### 데이터 읽기 및 이용데이터 수정

In [1]:
import pandas as pd
customer = pd.read_csv('customer_join.csv', encoding='cp949')
uselog_months = pd.read_csv('use_log_months.csv', encoding='cp949')

In [2]:
len(uselog_months)

36842

In [3]:
year_months = list(uselog_months["연월"].unique())

In [4]:
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [5]:
uselog_months.head()

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [6]:
uselog = pd.DataFrame()

# 2018.05부터 2019.03까지
for i in range(1, len(year_months)):
    # 각 연월 데이터만 tmp에 저장(예 : i == 1, 2018.05 데이터만)
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    # 칼럼명 변경 count -> count_0
    tmp = tmp.rename(columns={"count":"count_0"})
    # 1개월 전 데이터(2018.05일 경우 2018.04 데이터)를 tmp_before에 저장
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]]
    # 연월 칼럼 삭제
    del tmp_before["연월"]
    # 칼럼명 변경 count -> count_1
    tmp_before = tmp_before.rename(columns={"count":"count_1"})
    # 데이터 결합
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    # 만들어둔 데이터 프레임 변수에 결합
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [7]:
# 2018.04의 데이터가 빠진 모습
len(uselog)

33851

### 탈퇴전월의 탈퇴고객데이터 작성

In [8]:
customer[customer['is_deleted'] == 1].head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-04-30,23
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,1_주간,7500,2_일반,3.0,3.0,3,3,0,2018-04-30,23
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,1.0,1.0,1,1,0,2018-04-30,23
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,2_야간,6000,2_일반,3.5,3.5,4,3,1,2018-05-31,23
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-05-31,23


In [9]:
# dateutil.relativedelta : 날짜와 시간 간의 상대적인 차이를 계산할 수 있는 기능을 제공
from dateutil.relativedelta import relativedelta
# 탈퇴한 고객 데이터만 변수에 저장
exit_customer = customer.loc[customer["is_deleted"]==1]

In [10]:
# 칼럼을 만들고 거기에 None 값을 넣음
exit_customer
exit_customer.loc[:, "exit_date"] = None

C:\Users\YoonSunJun\AppData\Local\Temp\ipykernel_9244\2506978071.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer.loc[:, "exit_date"] = None


In [11]:
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,None
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,1_주간,7500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,None
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,1.0,1.0,1,1,0,2018-04-30,23,None
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,2_야간,6000,2_일반,3.5,3.5,4,3,1,2018-05-31,23,None
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-05-31,23,None


In [12]:
# end_date의 type 변경
exit_customer.loc[:, "end_date"] = pd.to_datetime(exit_customer["end_date"])
# 탈퇴한 회원 수 만큼 반복
for i in range(len(exit_customer)):
    # -1 = exit_date, 5 = end_customer / exit_date에 등록이 끝난 달의 전 달을 넣는다
    exit_customer.iloc[i, -1] = exit_customer.iloc[i, 5] - relativedelta(months=1)
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30 00:00:00,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30 00:00:00,CA1,1,1_주간,7500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00
730,HD542886,XX,C01,M,2016-05-01,2018-04-30 00:00:00,CA1,1,0_종일,10500,2_일반,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31 00:00:00,CA1,1,2_야간,6000,2_일반,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31 00:00:00,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00


In [13]:
# 연월에 등록기간이 끝난 날의 년도와 월을 넣는다
exit_customer.loc[:, "연월"] = pd.to_datetime(exit_customer.loc[:,"exit_date"]).dt.strftime("%Y%m")
# 연월의 type을 string으로 변경
uselog.loc[:, "연월"] = uselog.loc[:, "연월"].astype(str)
# 새로운 변수 만들고 이용로그와 exit_customer 데이터를 "customer_id", "연월" 기준에 맞춰 데이터 결합
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
# 탈퇴한 회원의 데이터만 넣어서 탈퇴하지 않은 회원의 데이터가 들어가있지 않은 모습이다.
exit_uselog.head()

33851


C:\Users\YoonSunJun\AppData\Local\Temp\ipykernel_9244\3144863483.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer.loc[:, "연월"] = pd.to_datetime(exit_customer.loc[:,"exit_date"]).dt.strftime("%Y%m")
C:\Users\YoonSunJun\AppData\Local\Temp\ipykernel_9244\3144863483.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['201805' '201805' '201805' ... '201903' '201903' '201903']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  uselog.loc[:, "연월"] = uselog.loc[:, "연월"].astype(str)


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# name에 결측지가 있는 데이터만 삭제
exit_uselog = exit_uselog.dropna(subset=["name"])
# 길이 확인
print(len(exit_uselog))
# 손님 id의 중복이 없는 지 확인
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30 00:00:00,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30 00:00:00,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30 00:00:00,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30 00:00:00,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30 00:00:00,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### 지속회원의 데이터 작성

In [15]:
# 변수 만들고 탈퇴하지 않은 회원 데이터만 저장
conti_customer = customer.loc[customer["is_deleted"]==0]
# 회원 id를 기준으로 이용로그와 지속회원 데이터를 결합
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
conti_uselog.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,2_야간,6000.0,2_일반,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201805,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201805,AS015233,7,NaN,XXXXX,C01,M,2018-05-13,NaN,CA2,...,0_종일,10500.0,0_입회비반액할인,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201805,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,0_종일,10500.0,2_일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201805,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,...,2_야간,6000.0,2_일반,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0


In [16]:
# uselog는 2018.05부터 데이터기 떄문에 샘플개수가 33851
print(len(conti_uselog))
# 이름에 결측치가 있는 데이터 삭제
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [17]:
# sample 메서드는 데이터프레임의 행이나 열에서 무작위 샘플을 추출할 수 있게 해줍니다.
# frac: 데이터프레임에서 추출할 샘플의 비율을 지정합니다. n 매개변수가 지정된 경우, frac는 무시됩니다.
# 예시: frac=0.5이면 데이터프레임의 50%를 무작위로 선택합니다.
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
# drop_duplicates 메서드는 데이터프레임에서 중복된 행을 제거하는 데 사용됩니다.
# subset: 중복을 판단할 열 또는 열 목록을 지정합니다. 이 매개변수를 사용하면 특정 열만 기준으로 중복을 제거할 수 있습니다.
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
# 길이 출력
print(len(conti_uselog))
# 손님 별로 랜덤으로 하나의 로그 데이터만 저장
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201901,OA358651,5,2.0,XXXXX,C01,F,2017-10-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.833333,6.5,9.0,2.0,1.0,2019-04-30,18.0
1,201812,IK686461,7,4.0,XXXX,C01,M,2016-12-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.583333,6.0,7.0,3.0,1.0,2019-04-30,28.0
2,201810,HD671982,3,5.0,XXXXX,C01,M,2016-05-01,NaN,CA1,...,0_종일,10500.0,2_일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,35.0
3,201810,IK750625,4,8.0,XX,C03,M,2017-09-01,NaN,CA1,...,2_야간,6000.0,2_일반,6.166667,6.0,9.0,4.0,1.0,2019-04-30,19.0
4,201808,OA228506,4,3.0,XXXXX,C01,M,2017-06-01,NaN,CA1,...,0_종일,10500.0,2_일반,6.250000,6.0,10.0,3.0,1.0,2019-04-30,22.0


In [18]:
# 예측 데이터, 지속회원과 탈퇴회원을 결합
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201901,OA358651,5,2.0,XXXXX,C01,F,2017-10-01,NaN,CA1,...,10500.0,2_일반,5.833333,6.5,9.0,2.0,1.0,2019-04-30,18.0,NaN
1,201812,IK686461,7,4.0,XXXX,C01,M,2016-12-01,NaN,CA1,...,10500.0,2_일반,5.583333,6.0,7.0,3.0,1.0,2019-04-30,28.0,NaN
2,201810,HD671982,3,5.0,XXXXX,C01,M,2016-05-01,NaN,CA1,...,10500.0,2_일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,35.0,NaN
3,201810,IK750625,4,8.0,XX,C03,M,2017-09-01,NaN,CA1,...,6000.0,2_일반,6.166667,6.0,9.0,4.0,1.0,2019-04-30,19.0,NaN
4,201808,OA228506,4,3.0,XXXXX,C01,M,2017-06-01,NaN,CA1,...,10500.0,2_일반,6.250000,6.0,10.0,3.0,1.0,2019-04-30,22.0,NaN


### 예측할 달의 재적기간을 작성

In [19]:
# 새 칼럼 생성 및 0 넣기
predict_data["period"] = 0

In [20]:
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period
0,201901,OA358651,5,2.0,XXXXX,C01,F,2017-10-01,NaN,CA1,...,2_일반,5.833333,6.5,9.0,2.0,1.0,2019-04-30,18.0,NaN,0
1,201812,IK686461,7,4.0,XXXX,C01,M,2016-12-01,NaN,CA1,...,2_일반,5.583333,6.0,7.0,3.0,1.0,2019-04-30,28.0,NaN,0
2,201810,HD671982,3,5.0,XXXXX,C01,M,2016-05-01,NaN,CA1,...,2_일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,35.0,NaN,0
3,201810,IK750625,4,8.0,XX,C03,M,2017-09-01,NaN,CA1,...,2_일반,6.166667,6.0,9.0,4.0,1.0,2019-04-30,19.0,NaN,0
4,201808,OA228506,4,3.0,XXXXX,C01,M,2017-06-01,NaN,CA1,...,2_일반,6.250000,6.0,10.0,3.0,1.0,2019-04-30,22.0,NaN,0


In [21]:
# 새 칼럼 생성 및 연월 데이터 타입 변경 밎 년도랑 월만 넣기
predict_data.loc[:, "now_date"] = pd.to_datetime(predict_data.loc[:, "연월"], format="%Y%m")
predict_data.loc[:, "start_date"] = pd.to_datetime(predict_data.loc[:, "start_date"])
# predict_data만큼 반복
for i in range(len(predict_data)):
    # -1 == 'now_date', 7 == 'start_date'
    delta = relativedelta(predict_data.iloc[i]['now_date'], predict_data.iloc[i]['start_date'])
    # -2 == 'period'
    predict_data.iloc[i, -2] = int(delta.years*12 + delta.months)
predict_data.head(10)

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201901,OA358651,5,2.0,XXXXX,C01,F,2017-10-01 00:00:00,NaN,CA1,...,5.833333,6.5,9.0,2.0,1.0,2019-04-30,18.0,NaN,15,2019-01-01
1,201812,IK686461,7,4.0,XXXX,C01,M,2016-12-01 00:00:00,NaN,CA1,...,5.583333,6.0,7.0,3.0,1.0,2019-04-30,28.0,NaN,24,2018-12-01
2,201810,HD671982,3,5.0,XXXXX,C01,M,2016-05-01 00:00:00,NaN,CA1,...,4.833333,5.0,7.0,3.0,1.0,2019-04-30,35.0,NaN,29,2018-10-01
3,201810,IK750625,4,8.0,XX,C03,M,2017-09-01 00:00:00,NaN,CA1,...,6.166667,6.0,9.0,4.0,1.0,2019-04-30,19.0,NaN,13,2018-10-01
4,201808,OA228506,4,3.0,XXXXX,C01,M,2017-06-01 00:00:00,NaN,CA1,...,6.250000,6.0,10.0,3.0,1.0,2019-04-30,22.0,NaN,14,2018-08-01
5,201811,HI263602,5,9.0,XXXXX,C02,F,2017-10-01 00:00:00,NaN,CA1,...,5.916667,6.0,9.0,3.0,1.0,2019-04-30,18.0,NaN,13,2018-11-01
6,201903,PL499733,4,7.0,XXXX,C02,F,2017-10-01 00:00:00,NaN,CA1,...,6.166667,6.0,11.0,3.0,1.0,2019-04-30,18.0,NaN,17,2019-03-01
7,201808,GD629291,8,10.0,XXX,C01,M,2018-07-04 00:00:00,NaN,CA1,...,8.333333,8.0,10.0,6.0,1.0,2019-04-30,9.0,NaN,0,2018-08-01
8,201901,PL913199,6,3.0,XXX,C01,F,2016-04-01 00:00:00,NaN,CA1,...,4.666667,4.5,7.0,3.0,1.0,2019-04-30,36.0,NaN,33,2019-01-01
9,201807,HI237883,6,5.0,XXXXX,C01,M,2018-01-01 00:00:00,NaN,CA1,...,6.500000,6.0,9.0,4.0,1.0,2019-04-30,15.0,NaN,6,2018-07-01


### 결측치 제거

In [22]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               267
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [23]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2627
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2627
period                  0
now_date                0
dtype: int64

### 문자열 변수를 처리할 수 있게 가공

In [24]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,2_일반,0_종일,F,2.0,1.0,15,0.0
1,2_일반,0_종일,M,4.0,1.0,24,0.0
2,2_일반,0_종일,M,5.0,1.0,29,0.0
3,2_일반,2_야간,M,8.0,1.0,13,0.0
4,2_일반,0_종일,M,3.0,1.0,14,0.0


In [25]:
# 켐페인, 클래스, 성별
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
0,2.0,1.0,15,0.0,False,False,True,True,False,False,True,False
1,4.0,1.0,24,0.0,False,False,True,True,False,False,False,True
2,5.0,1.0,29,0.0,False,False,True,True,False,False,False,True
3,8.0,1.0,13,0.0,False,False,True,False,False,True,False,True
4,3.0,1.0,14,0.0,False,False,True,True,False,False,False,True


In [26]:
# 할인 안 받은 일반 데이터 삭제
# 야간반 삭제
# 남자 데이터 삭제
del predict_data["campaign_name_2_일반"]
del predict_data["class_name_2_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
0,2.0,1.0,15,0.0,False,False,True,False,True
1,4.0,1.0,24,0.0,False,False,True,False,False
2,5.0,1.0,29,0.0,False,False,True,False,False
3,8.0,1.0,13,0.0,False,False,False,False,False
4,3.0,1.0,14,0.0,False,False,True,False,False


### 의사결정 나무를 사용해서 탈퇴예측모델을 작성

In [27]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

In [28]:
# 탈퇴 회원 데이터 저장
exit = predict_data.loc[predict_data["is_deleted"]==1]
# 지속 회원 데이터를 랜덤으로 탈퇴 회원 샘플 개수만큼 저장
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

In [29]:
# X는 두 데이터를 붙인 데이터
X = pd.concat([exit, conti], ignore_index=True)
# y는 탈퇴여부의 데이터
y = X["is_deleted"]
# X에 탈퇴여부 데이터 삭제
del X["is_deleted"]
# 훈련 데이터와 test 데이터 나누기
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

print(len(X_train), len(X_test))

1578 526


In [30]:
# 1개월 전, 매달 같은 요일에 왔는 지 여부, 등록기간, 켐페인 혜택을 받은 여부, 주간반 종일반 여부, 여자인지 여부
X_train.head()

,count_1,routine_flg,period,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
673,4.0,0.0,4,True,False,True,False,False
1834,8.0,1.0,16,False,False,False,True,True
1443,6.0,1.0,14,False,False,False,False,True
1862,8.0,1.0,19,False,False,True,False,False
189,9.0,0.0,0,True,False,False,True,True


In [31]:
y_train.head()

673     1.0
1834    0.0
1443    0.0
1862    0.0
189     1.0
Name: is_deleted, dtype: float64

In [32]:
# 의사결정나무 모델 객체 생성
model = DecisionTreeClassifier(random_state=0)
# 훈련
model.fit(X_train, y_train)
# X_test 데이터를 가지고 y_test를 예측
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0.
 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1.

In [33]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1358,0.0,0.0
1514,0.0,0.0
1651,0.0,1.0
1087,0.0,0.0
1476,0.0,0.0


### 예측 모델을 평가 및 모델 튜닝

In [34]:
# y_test와 y_pred 데이터가 맞는 데이터만 개수 count
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
# 맞힌 확률 계산
score_test = correct / data_count
# 88.4% 정도
print(score_test)

0.8840304182509505


In [35]:
# 과적합이 된 부분
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9771863117870723
0.8840304182509505


In [36]:
# 데이터 다시 섞기
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

In [38]:
# 다시 학습 및 모델 점수 확인
# 과적합 방지로 깊이를 5개로
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.9271229404309252
0.9125475285171103


### 모델에 기여하고 있는 변수를 확인

In [39]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.341802
1,routine_flg,0.100260
2,period,0.554421
3,campaign_name_0_입회비반액할인,0.003517
4,campaign_name_1_입회비무료,0.000000
5,class_name_0_종일,0.000000
6,class_name_1_주간,0.000000
7,gender_F,0.000000


### 회원 탈퇴 예측

In [41]:
# 임의로 회원 데이터 작성
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [42]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]

# 리스트 만들기
input_data = [count_1, routing_flg, period]
# 리스트 붙이기
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [43]:
input_data

[3, 1, 10, 0, 1, 1, 0, 0]

In [44]:
# 예측한 값
print(model.predict([input_data]))
# 0인지 1인지에 대한 확률
print(model.predict_proba([input_data]))

[1.]
[[0.01428571 0.98571429]]


c:\Users\YoonSunJun\anaconda3\envs\churn\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\YoonSunJun\anaconda3\envs\churn\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
